In [20]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import os
import json
from collections import Counter

In [45]:
fontane = 'https://textgridrep.org/search?query=fontane&order=relevance&limit=507&filter=format%3Atext%2Fxml&filter=work.genre%3Averse'
christen = 'https://textgridrep.org/search?query=christen&order=relevance&limit=259&mode=list&filter=edition.agent.value:Christen%2C+Ada&filter=format%3Atext%2Fxml&filter=work.genre%3Averse'
conradi = 'https://textgridrep.org/search?query=conradi&order=relevance&limit=108&mode=list&filter=edition.agent.value:Conradi%2C+Hermann&filter=format%3Atext%2Fxml&filter=work.genre%3Averse'
dahn = 'https://textgridrep.org/search?query=dahn&order=relevance&limit=209&mode=list&filter=edition.agent.value:Dahn%2C+Felix&filter=format%3Atext%2Fxml&filter=work.genre%3Averse'
keller = 'https://textgridrep.org/search?query=keller&order=relevance&limit=581&mode=list&filter=work.genre:verse&filter=edition.agent.value%3AKeller%2C+Gottfried&filter=format%3Atext%2Fxml'
wille = 'https://textgridrep.org/search?query=wille&order=relevance&limit=131&mode=list&filter=edition.agent.value:Wille%2C+Bruno&filter=work.genre%3Averse&filter=format%3Atext%2Fxml'
#nichts für Janitschek, Sudermann, Vischer !

In [60]:
series = pd.Series(dtype = str)
counter = 0

URL = wille
html = requests.get(URL).content
soup = BeautifulSoup(html, 'lxml')
theodor = soup.find_all('a', 'tgrep result_button -download')
print(len(theodor))

131


In [61]:
text_dic = {}
for t in theodor:
    text = []
    link = t.get('href')
    html = requests.get(link).content
    soup = BeautifulSoup(html, 'lxml')
    text.append(soup.find_all('l'))
    title = soup.find_all('title')[0].string
    text_dic[title] = text
text_dic

{'Regen': [[<l xml:id="tg19.3.2">Wohl strömt ein feuchter Segen</l>,
   <l xml:id="tg19.3.3">Und labt das atmende Thal ...</l>,
   <l xml:id="tg19.3.4">Du meiner Seele Regen,</l>,
   <l xml:id="tg19.3.5">Wann strömest du einmal?</l>]],
 'Ich und Du': [[<l xml:id="tg66.3.2">Wir hielten uns umschlungen;</l>,
   <l xml:id="tg66.3.3">Nachtodem hauchte mild,</l>,
   <l xml:id="tg66.3.4">Der Junimond durchblaute</l>,
   <l xml:id="tg66.3.5">Gebüsch und Grasgefild.</l>,
   <l xml:id="tg66.3.7">Ich staunte in die Landschaft;</l>,
   <l xml:id="tg66.3.8">Die lag so fremd. Doch klang</l>,
   <l xml:id="tg66.3.9">Geheim aus Sternenmeeren</l>,
   <l xml:id="tg66.3.10">Ein heimatlicher Sang.</l>,
   <l xml:id="tg66.3.12">Ich staunte in dein liebes,</l>,
   <l xml:id="tg66.3.13">Mondbleiches Angesicht/</l>,
   <l xml:id="tg66.3.14">Auf deiner Augen Grunde</l>,
   <l xml:id="tg66.3.15">Erglomm ein fremdes Licht.</l>,
   <l xml:id="tg66.3.17">Und dich auch sah ich staunen;</l>,
   <l xml:id="tg66.3.18

In [62]:
outpath = '../corpus/autoren_corpora/textgrid/'
for title, texte in text_dic.items():
    fontane_texte = ''
    for text in texte:
        for t in text:
            if t.string != None:
                fontane_texte += t.string
                fontane_texte += ' '
    print(title, ': ', fontane_texte, '\n', '\n')
    if os.path.exists(outpath + 'wille_' + title+'.txt'):
        txtFile = open(outpath + 'wille_' + title +'.txt', 'w')
        txtFile.write('')
        txtFile.close()
    txtFile = open(outpath + 'wille_' + title +'.txt', 'a')
    txtFile.write(fontane_texte)
    txtFile.close()

Regen :  Wohl strömt ein feuchter Segen Und labt das atmende Thal ... Du meiner Seele Regen, Wann strömest du einmal?  
 

Ich und Du :  Wir hielten uns umschlungen; Nachtodem hauchte mild, Der Junimond durchblaute Gebüsch und Grasgefild. Ich staunte in die Landschaft; Die lag so fremd. Doch klang Geheim aus Sternenmeeren Ein heimatlicher Sang. Ich staunte in dein liebes, Mondbleiches Angesicht/ Auf deiner Augen Grunde Erglomm ein fremdes Licht. Und dich auch sah ich staunen; Die Lippen zuckten stumm. So weh war unsre Liebe/ Wir ahnten wohl, warum. So weh/ ob Mund an Munde Auch süßen Taumel trank; So weh/ ob Aug in Auge Auch liebetief versank. Wir fühlten, Herz an Herzen, Wie ewig dich und mich Ein banger Abgrund scheidet/ Wir sind ja du und ich! Wir schluchzten auf/ vor Heimweh! Die Heimat liegt so weit, Dort hinter Sternenmeeren, Weit, in der Ewigkeit. Dort in der Heimat findet Dies bange Schmachten Ruh: Es fließen ineinander/ O selig/ ich und du.  
 

Die beiden Waldfeuer :  Waldfeu

In [10]:
with open('../corpus/deutsches.lyrik.korpus.v4.noduplicates.intactpoems.json', 'r') as myfile:
    data=myfile.read()

In [39]:
autoren_liste = ['Dahn', 'Christen', 'Conradi','Fontane', 'Janitschek', 'Keller', 'Sudermann', 'Vischer','Wille']
obj = json.loads(data)
texte = {}
for i in obj:
    for autor in autoren_liste:
        if autor in obj[i]['author']:
            name = str(obj[i]['author'].split(',')[0])+'_'+str(obj[i]['title'])
            string = ''
            for zeile in obj[i]['lines']:
                string += zeile
                string += ' '
            texte[name] = string
texte

{'Dahn_Ein Königsspiel': "»Mich gelüstet,« sprach der König, »Mich gelüstet, o Dadanes, Deiner einz'gen Frau Mandane, Die du mehr liebst – also sagt man – Als dein Leben: gib die Schlanke Mir zu meinen hundert Frauen: Gönnst dem König du dein Weib?« ",
 'Keller_Der Schein trügt': 'Nur wer es weiß mit Klugheit zu beginnen, In seinen tiefsten Keller sich zu stehlen, Sieht üppig feuchtes Unkraut dort verhehlen Von dicken Schlangen wahre Königinnen. ',
 'Keller_\n                                    ': 'Herrlich in der Maienzeit Blaut des Himmels Kläre – Halt zum Opferdienst bereit Nun die blanke Schere! ',
 'Dahn_Des Mönches Nachtlied': "In deinen Zügen malet Sich sel'ge Traurigkeit: Dein Auge widerstrahlet Gott und Unendlichkeit. ",
 'Fontane_4. Es kann die Ehre dieser Welt': "Wenn's deinem Innersten gebricht An echten Stolzes Stütze, Ob dann die Welt dir Beifall spricht, Ist all dir wenig nütze. ",
 'Conradi_\n                                 ': 'Der da schafft in der Zeiten Dämmerungssc

In [40]:
autoren = texte.keys()
autor_count = []
for i in autoren:
    autor_count.append(i.split('_')[0])

In [41]:
Counter(autor_count)

Counter({'Dahn': 150,
         'Keller': 218,
         'Fontane': 233,
         'Conradi': 78,
         'Wille': 11,
         'Christen': 29})

In [44]:
outpath = '../corpus/autoren_corpora/'
for i, v in texte.items():
    if os.path.exists(outpath + i +'.txt'):
        txtFile = open(outpath + i +'.txt', 'w')
        txtFile.write('')
        txtFile.close()
    txtFile = open(outpath + i +'.txt', 'a')
    txtFile.write(v)
    txtFile.close()